In [58]:
import cv2 as cv
import numpy as np
import os
import keras
import glob
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [6]:
DATASET_PATH = '../data/'

In [65]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(6, activation='softmax'))

optimizer = Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.build((128, 100, 100, 1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (128, 98, 98, 64)         640       
                                                                 
 batch_normalization_3 (Batc  (128, 98, 98, 64)        256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (128, 49, 49, 64)        0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (128, 49, 49, 64)         0         
                                                                 
 flatten_3 (Flatten)         (128, 153664)             0         
                                                                 
 dense_6 (Dense)             (128, 128)               

In [56]:
def load_images():
    images = []
    gestures = ['blank', 'ok', 'thumbsup', 'thumbsdown', 'fist', 'five']

    for gesture in gestures:
        path = DATASET_PATH + gesture
        files = glob.glob(path + '/*')
        for i in range(0, 1500):
            image = cv.imread(files[i])
            image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
            image = cv.resize(image, (100, 100))
            images.append(image)
        
        
    return images

def load_labels():
    labels = []
    
    for _ in range(1500):
        labels.append([1, 0, 0, 0, 0, 0])
    for _ in range(1500):
        labels.append([0, 1, 0, 0, 0, 0])
    for _ in range(1500):
        labels.append([0, 0, 1, 0, 0, 0])
    for _ in range(1500):
        labels.append([0, 0, 0, 1, 0, 0])
    for _ in range(1500):
        labels.append([0, 0, 0, 0, 1, 0])
    for _ in range(1500):
        labels.append([0, 0, 0, 0, 0, 1])
    
    return labels

In [57]:
X = np.asarray(load_images())
y = np.asarray(load_labels())
X.shape, y.shape

((9000, 100, 100), (9000, 6))

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [61]:
X_train = X_train.reshape(X_train.shape[0], 100, 100, 1)
X_test = X_test.reshape(X_test.shape[0], 100, 100, 1)
X_train.shape, X_test.shape

((7200, 100, 100, 1), (1800, 100, 100, 1))

In [66]:
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'loss: {loss}; accuracy: {accuracy}')

Epoch 1/10
57/57 [==============================] - 48s 829ms/step - loss: 2.8832 - categorical_accuracy: 0.8676 - val_loss: 0.4518 - val_categorical_accuracy: 0.9650
Epoch 2/10
57/57 [==============================] - 47s 834ms/step - loss: 0.1717 - categorical_accuracy: 0.9619 - val_loss: 0.0745 - val_categorical_accuracy: 0.9861
Epoch 3/10
57/57 [==============================] - 45s 784ms/step - loss: 0.1432 - categorical_accuracy: 0.9715 - val_loss: 0.0135 - val_categorical_accuracy: 0.9978
Epoch 4/10
57/57 [==============================] - 43s 754ms/step - loss: 0.1024 - categorical_accuracy: 0.9747 - val_loss: 0.0035 - val_categorical_accuracy: 0.9978
Epoch 5/10
57/57 [==============================] - 44s 767ms/step - loss: 0.1161 - categorical_accuracy: 0.9785 - val_loss: 0.0014 - val_categorical_accuracy: 0.9994
Epoch 6/10
57/57 [==============================] - 44s 779ms/step - loss: 0.1616 - categorical_accuracy: 0.9749 - val_loss: 0.0024 - val_categorical_accuracy: 0.998

In [67]:
model.save('./hand_recognition_model.keras')